In [ ]:
import pandas as pd

In [ ]:
wildfire_data = pd.read_csv("mapdata_wildfires.csv")

In [ ]:
wildfire_data.head()

In [ ]:
wildfire_data.columns

In [ ]:
wildfire_data_columns_to_drop = ['incident_is_final', 'incident_date_last_update', 'incident_administrative_unit',
       'incident_administrative_unit_url', 'incident_location', 'incident_containment', 'incident_control', 'incident_cooperating_agencies','incident_type', 'incident_id', 'incident_url', 'incident_dateonly_extinguished', 'is_active', 'calfire_incident', 'notification_desired' ]

In [ ]:
wildfire_data = wildfire_data.drop(wildfire_data_columns_to_drop, axis = 1)

In [ ]:
wildfire_data.head()

In [ ]:
wildfire_data.columns

In [ ]:
wildfire_data_column_order = ['incident_name', 'incident_county', 'incident_dateonly_created',
                             'incident_date_created','incident_date_extinguished',  'incident_latitude',
                             'incident_longitude', 'incident_acres_burned']

In [ ]:
wildfire_data = wildfire_data[wildfire_data_column_order]

In [ ]:
wildfire_data.head()

In [ ]:
wildfire_data.columns = ['Name', 'County', 'Date', 'Start Date and Time', 'End Date and Time', 'Latitude', 'Longitude', 'Acres Burned']

In [ ]:
wildfire_data.head()

In [ ]:
wildfire_data['Start Date and Time'] = pd.to_datetime(wildfire_data['Start Date and Time'])
wildfire_data['End Date and Time'] = pd.to_datetime(wildfire_data['End Date and Time'])
wildfire_data['Duration'] = (wildfire_data['End Date and Time'] - wildfire_data['Start Date and Time']) / pd.Timedelta('1 day')

In [ ]:
wildfire_data.head()

In [ ]:
len(wildfire_data[wildfire_data['Duration'] < 0])

In [ ]:
weather_data = pd.read_csv("weather_data_wildfires.csv")

In [ ]:
weather_data.head()

In [ ]:
weather_data.columns

In [ ]:
weather_data_columns_to_drop = ['Wind Chill', 'Heat Index','Precipitation Cover', 'Snow', 'Snow Depth', 
                                'Visibility','Sea Level Pressure',  'Weather Type', 'Info', 'Conditions']

In [ ]:
weather_data = weather_data.drop(weather_data_columns_to_drop, axis = 1)

In [ ]:
weather_data.head()

In [ ]:
drought_index_data = pd.read_csv("drought_data_wildfires.csv")
vegatation_index_data = pd.read_csv("vegetation_indices_data_wildfires.csv")
land_cover_data = pd.read_csv("land_cover_data_wildfires.csv")

In [ ]:
merged_data_1 = pd.merge(wildfire_data, weather_data, on=["Latitude", "Longitude", "Date"], how="inner")


merged_data_2 = pd.merge(merged_data_1, drought_index_data, on=["Latitude", "Longitude", "Date"], how="inner")


merged_data_3 = pd.merge(merged_data_2, vegatation_index_data, on=["Latitude", "Longitude", "Date"], how="inner")


merged_data_final = pd.merge(merged_data_3, land_cover_data, on=["Latitude", "Longitude", "Date"], how="inner")


merged_data_final.head()

In [ ]:
elevation_slope_data = pd.read_csv("elevation_slope_data_wildfires.csv")

In [ ]:
elevation_slope_data.head()

In [ ]:
elevation_slope_data.columns = ["Latitude", "Longitude","Elevation","Slope"]

In [ ]:
merged_data_final2 = pd.merge(merged_data_final, elevation_slope_data, on=["Latitude", "Longitude"], how="inner")

In [ ]:
merged_data_final2.head()

In [ ]:
merged_data_final2.reset_index(drop=True, inplace=True)

In [ ]:
merged_data_final2.info()

In [ ]:
merged_data_final2['Land Cover'] = merged_data_final2['Land Cover'].astype('category')

In [ ]:
merged_data_final2.info()

In [ ]:
merged_data_final2.describe()

In [ ]:
# Outliers in the Acres Burned column based on the z score method

from scipy import stats
import numpy as np

z_scores = np.abs(stats.zscore(merged_data_final2['Acres Burned']))
outliers = np.where(z_scores > 2)

print(outliers)

outlier_values = merged_data_final2['Acres Burned'].iloc[outliers]
print(outlier_values)

In [ ]:
merged_data_final2 = merged_data_final2.drop_duplicates()
merged_data_final2.info()

In [ ]:
merged_data_final2.to_csv("merged_dataset_not_cleaned.csv", index = False)

In [ ]:
merged_data_final2.head()

In [ ]:
merged_data_final2.describe()